For this first deliverable, I need to develop this:
<br /> 1.- Apply scaling techniques
<br /> 2.- preprocess of data

In [2]:
# All the imports come here
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import ImageDataGenerator

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# It's important to comment the import depending on the OS
# Windows PC
base_dir = r"/dataset/car parts"

# Mac OS
# fill here xd

# Google Colab
# fill here xd

In [ ]:
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")

# I will omit the test directory for now. I don't want to mix the test data with the training data

In [ ]:
# Time to preprocess the data

# First, scale pixels from 0-255 to 0-1
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# I don't want to augment the validation data nor the test data

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=10,
    class_mode="categorical",
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=10,
    class_mode="categorical", 
)

In [ ]:
# Just to check if the data is loading correctly
# Let's visualize some images from the training set

def visualize_data(generator, num_images=5):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        img, label = generator.next()
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(img[0])
        plt.title(f"Label: {label[0]}")
        plt.axis("off")
    plt.show()

# also some images from the validation set
def visualize_data_valid(generator, num_images=5):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        img, label = generator.next()
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(img[0])
        plt.title(f"Label: {label[0]}")
        plt.axis("off")
    plt.show()